In [1]:
import pdfplumber
import pandas as pd
import re

In [2]:
counties = ["Kosciusko", "Whitley"]

In [108]:
i = 1
county = counties[i]
county

'Whitley'

In [109]:
input_file = "../data-raw/in_2024_results/" + county + ".pdf"
output_file = "../data/in_2024_results/" + county + ".csv"
output_2 = "C:/Users/johnsor/Documents/openelections-data-in/2024/counties/20241105__in__general__" + str.lower(county) + "__precinct.csv"
output_2

'C:/Users/johnsor/Documents/openelections-data-in/2024/counties/20241105__in__general__whitley__precinct.csv'

In [110]:
with pdfplumber.open(input_file) as pdf:
    lines = [[line['text'] for line in page.extract_text_lines()] for page in pdf.pages]

In [111]:
p2 = lines[1]
p2[:30]

['11/5/2024 - 7:49:01 PM',
 'Election Date: 11/5/2024 Precinct Summary Report',
 'EMS',
 'Whitley County, Indiana',
 '2024 General Election',
 'INWHIG24',
 '11/5/2024',
 'E - # Of Election Day 271 PRECINCT STATUS: REPORTED',
 'A - # Of Paper Absentee 26',
 'W - # Of Walk-In Absentee 231',
 'PUBLIC COUNT: 528',
 'VOTER TURNOUT: 67.95%',
 'REGISTERED VOTERS: 777',
 'Precinct ID: 01',
 'Precinct Name: Cleveland NORTHEAST',
 '- - - - - - - - - - - - - - - - - - VOTES - - - - - - - - - - - - - - - -',
 'E A W TOTAL %',
 'VOTE FOR 1',
 'Public Question-Const Amendment',
 'VOTES=466',
 '158 6 109 273 58.58% Yes',
 '91 13 89 193 41.42% No',
 'VOTE FOR 1',
 'President and Vice-President of the U.S.',
 'VOTES=523',
 '210 17 169 396 75.72% (R) Trump \\ Vance',
 '50 9 54 113 21.61% (D) Harris \\ Walz',
 '3 0 3 6 1.15% (L) Oliver \\ Ter Maat',
 '5 0 3 8 1.53% (WTP) Kennedy Jr. \\ Shanahan',
 '0 0 0 0 0.00% Write-In']

In [112]:
line = p2[20]
print(line)
if re.search(r'^E - #', line):
    test = re.findall(r'Election Day (\d+) PRECINCT', line)[0]
elif re.search(r'^A - #', line):
    test = re.findall(r'Of Paper Absentee (\d+)', line)[0]
elif re.search(r'^W - #', line):
    test = re.findall(r'Walk-In Absentee (\d+)', line)[0]
elif re.search(r'Representative District (\d+)', line):
    print(re.findall(r'District (\d+)', line)[0])
elif re.search(r'^Precinct Name', line):
        precint = re.findall(r'Precinct Name: (.*)', line)[0]
        print(line)
        print(precint)
elif re.search(r'\d%', line) and not meta: #Lines with percentage numbers that are not metadata are the values we want
            print(line.split(sep=' ', maxsplit=6)) #The line is a row of a table of 6 columns
line.split(' District ')

158 6 109 273 58.58% Yes
['158', '6', '109', '273', '58.58%', 'Yes']


['158 6 109 273 58.58% Yes']

In [113]:
cols = {'county' : [],
        'precinct' : [],
        'office' : [],
        'district' : [],
        'party' : [],
        'candidate' : [],
        'election_day' : [],
        'absentee' : [],
        'early_voting' : [],
        'votes' : []}

In [114]:
for p in lines:
    vote = False #Signifies when a "VOTE FOR ..." line has occurred
    meta = True #Signifies when lines are part of metadata of the precinct
    for (lineno, line) in enumerate(p): #for each line of the page
        if re.search('E - #', line):
            election_day = re.findall(r'Election Day (\d+) PRECINCT', line)[0]
            meta = True
        elif re.search(r'^A - #', line):
            absentee = re.findall(r'Of Paper Absentee (\d+)', line)[0]
        elif re.search(r'^W - #', line):
            early_voting = re.findall(r'Walk-In Absentee (\d+)', line)[0]
        elif re.search(r'^PUBLIC COUNT', line):
            votes = re.findall(r'PUBLIC COUNT: (\d+)', line)[0]
        elif re.search(r'^REGISTERED VOTERS', line):
            registered = re.findall(r'REGISTERED VOTERS: (\d+)', line)[0]
        elif re.search(r'^Precinct Name', line):
            precinct = re.findall(r'Precinct Name: (.*)', line)[0]
            cols['county'].extend([county, county])
            cols['precinct'].extend([precinct, precinct])
            cols['office'].extend(['Ballots Cast', 'Registered Voters'])
            cols['district'].extend(['',''])
            cols['party'].extend(['',''])
            cols['candidate'].extend(['',''])
            cols['election_day'].extend([election_day,''])
            cols['absentee'].extend([absentee,''])
            cols['early_voting'].extend([early_voting,''])
            cols['votes'].extend([votes, registered])
        elif re.search('VOTE FOR', line):#The 'VOTE FOR..." line means the next line is what is being voted on
            vote = True
        elif vote: #If the 'VOTE FOR..." line occured earlier
            vote = False #Reset this
            meta = False #This line means we are not in meta dinner anymore
            if re.search(r'Representative District (\d+)', line):
                office, district = line.split(' District ')
            else:
                office = line
                district = ''
        elif re.search(r'\d%', line) and not meta: #Lines with percentage numbers that are not metadata are the values we want
            E, A, W, V, _, option = line.split(maxsplit=5) #The line is a row of a table of 6 columns
            if re.search(r'\(\w\)', option):
                P = re.findall(r'\((\w)\)', option)[0]
                option = re.findall(r'\(\w\) (.*)', option)[0]
            else:
                P = ''
            cols['county'].append(county)
            cols['precinct'].append(precinct)
            cols['office'].append(office)
            cols['district'].append(district)
            cols['party'].append(P)
            cols['candidate'].append(option)
            cols['election_day'].append(E)
            cols['absentee'].append(A)
            cols['early_voting'].append(W)
            cols['votes'].append(V)

In [115]:
df = pd.DataFrame(data=cols)
df

county             precinct                                    office  \
0     Whitley  Cleveland NORTHEAST                              Ballots Cast   
1     Whitley  Cleveland NORTHEAST                         Registered Voters   
2     Whitley  Cleveland NORTHEAST           Public Question-Const Amendment   
3     Whitley  Cleveland NORTHEAST           Public Question-Const Amendment   
4     Whitley  Cleveland NORTHEAST  President and Vice-President of the U.S.   
...       ...                  ...                                       ...   
1774  Whitley     Washington SOUTH                    IN Supreme Court-Massa   
1775  Whitley     Washington SOUTH                   IN Supreme Court-Molter   
1776  Whitley     Washington SOUTH                   IN Supreme Court-Molter   
1777  Whitley     Washington SOUTH          Court of Appeals Dist 4-Pyle III   
1778  Whitley     Washington SOUTH          Court of Appeals Dist 4-Pyle III   

     district party      candidate election_day absentee early_voting votes  
0                                           271       26          231   528  
1                                                                       777  
2                              Yes          158        6          109   273  
3                               No           91       13           89   193  
4                 R  Trump \ Vance          210       17          169   396  
...       ...   ...            ...          ...      ...          ...   ...  
1774                            No           31        2           22    55  
1775                           Yes          127        6           59   192  
1776                            No           32        2           23    57  
1777                           Yes          132        7           62   201  
1778                            No           29        1           21    51  

[1779 rows x 10 columns]

In [116]:
df.to_csv(output_file)

In [117]:
df.to_csv(output_2)